# Notebook 1 : Bronze - Import et première vue (Spark SQL)

**Durée** : 10 minutes  
**Lakehouse** : Lakehouse_bronze  
**Objectif** : Charger les données brutes et identifier les problèmes de qualité

## Cellule 1 : Configuration et vérification Spark

In [ ]:
# Vérifier que Spark est disponible
print(f"✅ Spark version: {spark.version}")
print(f"✅ Lakehouse par défaut: Lakehouse_bronze")

## Cellule 2 : Import consumption_raw.csv

In [ ]:
# Charger le CSV avec PySpark
df_consumption = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("Files/data_bronze_notebooks/consumption_raw.csv")

# Créer la table dans le schéma bronze
df_consumption.write.mode("overwrite").format("delta").saveAsTable("bronze.consumption_raw")

print(f"✅ Table bronze.consumption_raw créée : {df_consumption.count()} lignes")

## Cellule 3 : Première vue des données (voir le bordel)

In [ ]:
%%sql
SELECT * FROM bronze.consumption_raw LIMIT 20

## Cellule 4 : Détection des problèmes de qualité

In [ ]:
%%sql
-- Compter les NULL, codes erreur, doublons
SELECT 
    COUNT(*) as total_rows,
    COUNT(DISTINCT timestamp, site_id) as unique_rows,
    COUNT(*) - COUNT(DISTINCT timestamp, site_id) as duplicates,
    SUM(CASE WHEN consumption_mw IS NULL THEN 1 ELSE 0 END) as null_values,
    SUM(CASE WHEN consumption_mw < 0 THEN 1 ELSE 0 END) as error_codes,
    SUM(CASE WHEN consumption_mw > 10 THEN 1 ELSE 0 END) as outliers
FROM bronze.consumption_raw

## Cellule 5 : Import des autres fichiers

In [ ]:
# Market prices
spark.read.csv("Files/data_bronze_notebooks/market_prices.csv", header=True, inferSchema=True) \
    .write.mode("overwrite").format("delta").saveAsTable("bronze.market_prices")

# Weather
spark.read.csv("Files/data_bronze_notebooks/weather_data.csv", header=True, inferSchema=True) \
    .write.mode("overwrite").format("delta").saveAsTable("bronze.weather_data")

# Sites reference
spark.read.csv("Files/data_bronze_notebooks/sites_reference.csv", header=True, inferSchema=True) \
    .write.mode("overwrite").format("delta").saveAsTable("bronze.sites_reference")

# Maintenance events
spark.read.csv("Files/data_bronze_notebooks/maintenance_events.csv", header=True, inferSchema=True) \
    .write.mode("overwrite").format("delta").saveAsTable("bronze.maintenance_events")

print("✅ Toutes les tables Bronze créées")

## Cellule 6 : Vérification rapide de toutes les tables

In [ ]:
%%sql
SELECT 'consumption' as table_name, COUNT(*) as rows FROM bronze.consumption_raw
UNION ALL
SELECT 'prices', COUNT(*) FROM bronze.market_prices
UNION ALL
SELECT 'weather', COUNT(*) FROM bronze.weather_data
UNION ALL
SELECT 'sites', COUNT(*) FROM bronze.sites_reference
UNION ALL
SELECT 'maintenance', COUNT(*) FROM bronze.maintenance_events

## Cellule 7 : Résumé

### ✅ Bronze import terminé

**Données chargées** :
- ~18,144 lignes de consommation (dont ~900 doublons, ~540 NULL, ~360 codes erreur)
- 2,880 lignes de prix spot
- 720 lignes météo
- 6 sites
- 8 événements maintenance

**Problèmes détectés** :
- ❌ 5% doublons
- ❌ 3% valeurs NULL
- ❌ 2% codes erreur (-999, -888, -777)
- ❌ Formats de dates multiples
- ❌ Valeurs aberrantes (> 10 MW)

➡️ **Prochaine étape** : Nettoyage dans Silver (Notebook 2)